In [1]:
import requests 
from bs4 import BeautifulSoup 


def addUrls(url, listUrl): 
    #open with GET method 
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
        listUrl.append(url)
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    l=soup.find("a",{"class":"_2glka"})   #_2glka has to be adjusted based on website  
    if (l.get('href') == ""):
        print('Done adding')
    else:                
        addUrls(l.get('href'), listUrl) 





In [2]:
url='https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB'
listUrl = []
addUrls(url, listUrl)

Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Successfully opened the web page
Done adding


In [3]:
print(listUrl)

['https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB', 'https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB&page=2', 'https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB&page=3', 'https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB&page=4', 'https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIojO_5sRmhgPWEX1pbm_qIdlzgtOgyk2fIQFgbeB5jP695hoaAlGJEALw_wcB&page=5', 'https://www.apartmentguide.com/apartments/Ohio/Dayton/?WT.mc_id=21000&gclid=Cj0KCQjws5HlBRDIARIsAOomqA1pP7zIoj

In [3]:
### Some apartments do not have ratings, in this case, NA


def searchApartmentProperties(url, f):
    global oneTime
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    l2=soup.findAll("div",{"data-tid":"listing-info"})
    if(oneTime):
        for prop in l2[1].findAll('div'):
            if(prop.get('data-tid')):
                f.write(prop.get('data-tid')+'#')
        oneTime = False
        f.write('\n')
    index = 0
    done = False
    for listing in l2:    
        for i in listing.findAll('div'):
            #ulaText column is missing or ratings column is missing
            if (index == 4 and i.get('data-tid')=='phone'):
                f.write('NA'+'#'+'NA'+'#')
            if ((index == 5 and done == False) and i.get('data-tid')=='phone'):
                f.write('NA'+'#')                  
            if (index == 4 and i.get('data-tid')=='ratings'):
                f.write('NA'+'#')        
                done = True
     
            
            if(i.get('data-tid')):
                print(i.get('data-tid'))
                print(i.text)
                f.write(i.text+'#')
                index = index + 1
        done = False
        index = 0
        print()
        f.write('\n')

In [4]:
i = 1
f = open("test.txt", "a")
oneTime = True
for url in listUrl:
    print('page %.f'%i)  
    searchApartmentProperties(url, f)   
    i = i + 1
f.close()

page 1
Successfully opened the web page
price
$880+
propertyname
Water Street Apartments
bedroom
1–3 Beds
bathroom
1–2 Baths
ulaText
Only 1 unit left
ratings
(1)
phone
(937) 401-2591

price
$899+
propertyname
The Landing
bedroom
1–3 Beds
bathroom
1–2 Baths
ulaText
10+ Units Available
ratings
(2)
phone
(937) 660-6044

price
$800+
propertyname
Walden Village
bedroom
1–3 Beds
bathroom
1–2 Baths
ulaText
7 units available
phone
(937) 912-4432

price
$619+
propertyname
The Wynds
bedroom
1–2 Beds
bathroom
1 Bath
ratings
(2)
phone
(844) 629-1867

price
$659+
propertyname
The Commons at Kettering
bedroom
1–3 Beds
bathroom
1 Bath
ratings
(5)
phone
(937) 490-9417

price
$865+
propertyname
Lofts At Willow Creek
bedroom
1–2 Beds
bathroom
1–2 Baths
ratings
(9)
phone
(937) 912-4768

price
$561+
propertyname
Cobblegate Apartments
bedroom
2–3 Beds
bathroom
1 Bath
ratings
(1)
phone
(937) 412-3743

price
$740+
propertyname
Washington Place
bedroom
1–3 Beds
bathroom
1–2 Baths
ulaText
4 units left
ratings


Successfully opened the web page
price
$880+
propertyname
Water Street Apartments
bedroom
1–3 Beds
bathroom
1–2 Baths
ulaText
Only 1 unit left
ratings
(1)
phone
(937) 401-2591

price
$1,200+
propertyname
First Place Luxury Apartments
bedroom
2–3 Beds
bathroom
2 Baths
phone
(937) 329-9958

price
$672+
propertyname
Creekside Villas
bedroom
2–3 Beds
bathroom
1 Bath
ratings
(2)
phone
(937) 949-5062

price
$799+
propertyname
Stonecreek Apartments
bedroom
2–3 Beds
bathroom
1 Bath
ulaText
4 units left
ratings
(23)
phone
(844) 606-7360

price
$669+
propertyname
Dovetree Apartments
bedroom
1–2 Beds
bathroom
1–2 Baths
ratings
(4)
phone
(937) 630-4868

price
$599+
propertyname
Timber Ridge
bedroom
2–3 Beds
bathroom
1 Bath
phone
(937) 660-6566

price
$499+
propertyname
Stillwater Park Apartments
bedroom
1–3 Beds
bathroom
1 Bath
ratings
(9)
phone
(937) 203-3808

price
$539+
propertyname
Van Buren Village
bedroom
1–2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 412-3501

price
$1,050+
p

Successfully opened the web page
price
$915+
propertyname
The Promenade At Beavercreek
bedroom
1–2 Beds
bathroom
1–2 Baths
ratings
(14)
phone
(937) 912-4496

price
$625
propertyname
57 Ernst Ave
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 435-5000

price
$850
propertyname
38 Willowwood Dr
bedroom
3 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 303-8769

price
$850
propertyname
840 Creighton Ave
bedroom
3 Beds
bathroom
2 Baths
ulaText
Only 1 unit left
phone
(937) 702-9044

price
$725
propertyname
2048 Stegman Ave
bedroom
3 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(513) 737-2640

price
$595
propertyname
50 Willowwood Dr
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 617-4345

price
$725
propertyname
916 Linden Ave
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(513) 737-2640

price
$500
propertyname
922 Linden Ave
bedroom
1 Bed
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(720) 630-9908

price
$500
prope

Successfully opened the web page
price
$789+
propertyname
Fieldstone
bedroom
1–3 Beds
bathroom
1–2 Baths
ratings
(7)
phone
(937) 912-4497

price
$1,000
propertyname
108 Ashwood Ave
bedroom
4 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 979-4693

price
$895
propertyname
229 Bellaire Ave
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(513) 737-2640

price
$775
propertyname
3712 Wayne Ave
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(513) 737-2640

price
$600
propertyname
1988 Victoria Ave
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 839-2176

price
$600
propertyname
1704 Huffman Ave
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 490-9580

price
$600
propertyname
1910 Colonial Village Ln
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 839-2176

price
$470
propertyname
608 Hampshire Rd
bedroom
1 Bed
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 262-7083

price
$550
propertyna

Successfully opened the web page
price
$745+
propertyname
Oakwood Apartments
bedroom
2 Beds
bathroom
2 Baths
ulaText
Only 3 units left
ratings
(8)
phone
(937) 550-6097

price
$602
propertyname
4500 Waynedale Cir
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 259-9818

price
$602
propertyname
4530 Bufort Blvd
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 259-9818

price
$625
propertyname
5353 Dunmore Dr
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 259-9818

price
$1,250
propertyname
5341 Naples Dr
bedroom
4 Beds
bathroom
2 Baths
ulaText
Only 1 unit left
phone
(937) 303-8769

price
$1,350
propertyname
4840 Powell Rd
bedroom
3 Beds
bathroom
3 Baths
ulaText
Only 1 unit left
phone
(937) 262-7083

price
$625
propertyname
7837 N Main St
bedroom
2 Beds
bathroom
1 Bath
ulaText
Only 1 unit left
phone
(937) 617-4345

price
$1,595
propertyname
4440 Glenheath Dr
bedroom
3 Beds
bathroom
2 Baths
ulaText
Only 1 unit left
phone
(937) 259

Successfully opened the web page
price
$915+
propertyname
The Promenade At Beavercreek
bedroom
1–2 Beds
bathroom
1–2 Baths
ratings
(14)
phone
(937) 912-4496

price
Contact Property for Price
propertyname
Ahepa One Apartments

price
Price Unavailable
propertyname
Shiloh Spring Court

price
Contact Property for Price
propertyname
Creekside Villas

price
Contact Property for Price
propertyname
Woodsview Place

price
Contact Property for Price
propertyname
Willows

price
Contact Property for Price
propertyname
Covenant Manor

price
Contact Property for Price
propertyname
ORCHARD GLEN

price
Price Unavailable
propertyname
Bloomfield Apartments
ratings
(3)

price
Price Unavailable
propertyname
Stonebridge Apartment Homes
ratings
(1)

price
Price Unavailable
propertyname
Fox Hunt

price
Price Unavailable
propertyname
Carriage Trails

price
Price Unavailable
propertyname
Stone Manor

price
Price Unavailable
propertyname
Miami Bluffs and King Richard

price
Price Unavailable
propertyname
Spring

In [5]:
def deleteChar(inputFile, outputFile, myChar):
    with open(inputFile, "rt") as fin:
        with open(outputFile, "wt") as fout:
            for line in fin:
                fout.write(line.replace(myChar, ''))

In [6]:
import os
deleteChar('test.txt', 'subList.txt', '(')
deleteChar('subList.txt', 'finalList.txt', ')')
os.remove("subList.txt")

In [7]:
import csv

with open('finalList.txt', 'r') as infile, open('final.csv', 'w') as outfile:
    stripped = (line.strip() for line in infile)
    lines = (line.split("#") for line in stripped if line)
    writer = csv.writer(outfile)
    writer.writerows(lines)